In [30]:
import json
from collections import Counter
import numpy as np
import pandas as pd
import os
import sys
import argparse
from os.path import dirname, realpath

import hashlib
import datetime
import datetime as datetime
sys.path.append("./src/")
#from disrisknet.utils.parsing import parse_args
import torch
from torch import nn, optim
from torch.nn import functional as F

In [14]:
import matplotlib
import pickle as pkl
import pprint as pp
from collections import OrderedDict
matplotlib.use('Agg')
import matplotlib.pyplot as plt



def print_stats(stats_fname):
    with open(stats_fname, 'rb') as f:
        stats = pkl.load(f)
    print("\nAUROCS:")
    aurocs = OrderedDict({k: stats[k] for k in stats if k[-7:] == 'auroc_c'})
    pp.pprint(aurocs)
    print("\nAUPRCS:")
    auprcs = OrderedDict({k: stats[k] for k in stats if k[-7:] == 'auprc_c'})
    pp.pprint(auprcs)

def get_params(result_path):
    with open(result_path, 'rb') as f:
        results = pkl.load(f)
    params = ['model_name', 'num_layers', 'num_heads', 'exclusion_interval', 'init_lr', "weight_decay", 'pool_name', 
    'no_random_sample_eval_trajectories', 'pad_size', 'epochs', 'train', 'dev', 'test', 'cuda', 
    'train_batch_size', 'eval_batch_size', 'max_batches_per_train_epoch', 'max_batches_per_dev_epoch', 
    'max_events_length', 'max_eval_indices', 'eval_auroc', 'eval_auprc', 'eval_c_index']
    configs = [{param: results[param]} for param in params]
    return configs

def plot_train_metric(epoch_stats_path, metric_keys):
    with open(epoch_stats_path, 'rb') as f:
        epoch_stats = pkl.load(f)
    plt.figure(figsize=(10, 7))
    lines = []
    labels = []
    for metric_key in metric_keys:
        # print(cls)
        # print(fprs[cls])
        num_of_epoch = len(epoch_stats[metric_key])
        line, = plt.plot(list(range(1, num_of_epoch + 1)), epoch_stats[metric_key], lw=1)
        lines.append(line)
        labels.append(metric_key)
    plt.xticks(np.arange(0, 21, 1))
    plt.xlim([0, 21])
    plt.xlabel('Epoch', size=20)
    plt.ylabel('Score', size=20)
    plt.legend(lines, labels, loc='best', prop=dict(size=12))
    plt.title("Training process for transformer model", size=16)

In [2]:
sys.path.append(dirname(dirname(realpath(os.getcwd()))))


In [3]:
src_path = "G:\\FillmoreCancerData\\markhe\\VTERisk" 
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 
def md5(key):
    return hashlib.md5(repr(key).encode()).hexdigest()

testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))
pat_ids = (testDF['patient_id'] ).astype(int)

testDF['pids'] = pat_ids.apply(md5)

In [4]:
logpath = os.path.join(src_path ,'logs_transformer_vte/H/11_7/', "d93e7956f15305ad23c78d6884ca2816.results.test_preds")
#logpath = os.path.join(src_path ,'logs_transformer_vte/H/11_13/', "f93477981e1e440dfe90d16d4e375622.results.test_preds")

In [5]:
tdf = pd.DataFrame({'patient_id': (testDF['patient_id'] ),    
                    'pids': (testDF['pids'] ),    
                  'dob': (testDF['dob'] ),  
                  'outcome_date': (testDF['outcome_date'] ), 
                  'obs_time_end': (testDF['obs_time_end'] ),  
                  'index_date': (testDF['index_date'] ),  
                  'diag_date': (testDF['diag_date'] ),  
                  'outcome': (testDF['outcome'])    })

with open(logpath, 'rb') as f:
    R = pickle.load(f)
    p = np.array(R['probs'])

Df = pd.DataFrame.from_dict(R)
Df['probs'] = Df['probs'].astype(float)
Df['exams'] = Df['exams'].astype(int)
M = pd.merge(tdf, Df)
 

In [31]:
modelpath = os.path.join(src_path ,'logs_transformer_vte/H/11_7/', "d93e7956f15305ad23c78d6884ca2816.results.test_preds")

with open(logpath, 'rb') as f:
    R = pickle.load(f)
    probs = (R['probs'])
    golds = (R['golds'])


In [7]:
from sklearn.calibration import CalibratedClassifierCV

In [8]:
modelpath =  os.path.join(src_path ,'snapshot_vte/H', "model_d93e7956f15305ad23c78d6884ca2816_model.pt")


In [10]:
#https://github.com/thunder001/VTERisk/blob/for_paper/src/scripts/summarizer/dev_analysis.py

'G:\\FillmoreCancerData\\markhe\\VTERisk\\snapshot_vte/H\\model_d93e7956f15305ad23c78d6884ca2816_model.pt'

In [12]:
def get_training_summary(model, log_dir, test = False):
    result_fname = model + '.results'
    result_path = os.path.join(log_dir, result_fname)
    dev_stat_path = result_path + '.dev_stats'
    params = get_params(result_path)
    pp.pprint(params)
    print_stats(dev_stat_path)
    if test:
        test_stat_path = result_path + '.test_stats'
        print_stats(test_stat_path)
    plot_train_metrics(result_path, model, metrics)

In [ ]:
modelpath = os.path.join(src_path ,'logs_transformer_vte/H/11_7/', "d93e7956f15305ad23c78d6884ca2816.results.test_preds")


In [17]:
result_path = os.path.join(src_path ,'logs_transformer_vte/H/11_7/d93e7956f15305ad23c78d6884ca2816.results')

In [18]:
params = get_params(result_path)


In [19]:
params

[{'model_name': 'transformer'},
 {'num_layers': 1},
 {'num_heads': 8},
 {'exclusion_interval': 0},
 {'init_lr': 0.00125},
 {'weight_decay': 0.001},
 {'pool_name': 'Softmax_AttentionPool'},
 {'no_random_sample_eval_trajectories': False},
 {'pad_size': 300},
 {'epochs': 18},
 {'train': True},
 {'dev': True},
 {'test': True},
 {'cuda': True},
 {'train_batch_size': 256},
 {'eval_batch_size': 128},
 {'max_batches_per_train_epoch': 2000},
 {'max_batches_per_dev_epoch': 1000},
 {'max_events_length': 400},
 {'max_eval_indices': 1},
 {'eval_auroc': True},
 {'eval_auprc': True},
 {'eval_c_index': False}]

In [21]:
test_path = os.path.join(src_path ,'logs_transformer_vte/H/11_7/d93e7956f15305ad23c78d6884ca2816.results.test_stats')

In [22]:
print_stats(test_path)



AUROCS:
OrderedDict([('test_3day_auroc_c', [0.7376350696334619])])

AUPRCS:
OrderedDict([('test_3day_auprc_c', [0.04663782661458212])])


In [23]:
epoch_path = os.path.join(src_path ,'logs_transformer_vte/H/11_7/d93e7956f15305ad23c78d6884ca2816.results.epoch_stats')

In [24]:
with open(epoch_path, 'rb') as f:
    epochs = pkl.load(f)

In [ ]:

# The first model trained from full VA dataset
result_path = 'logs/all_epoch_10_w1_batch_12816_devlen10.results'
params = get_params(result_path)
test_path = 'logs/all_epoch_10_w1_batch_12816_devlen10.results.test_stats'
print_stats(test_path)
epoch_path = 'logs/all_epoch_10_w1_batch_12816_devlen10.results.epoch_stats'
with open(epoch_path, 'rb') as f:
    epochs = pkl.load(f)

In [28]:
def compute_calibration_curve(golds_for_eval,Probs_for_eval):
    prob_true, prob_pred = calibration_curve(golds_for_eval, Probs_for_eval, n_bins=10)
    return prob_true, prob_pred


In [29]:
import sys
import matplotlib
import matplotlib.pyplot as plt
import argparse
import pickle as pkl
matplotlib.use('Agg')
import pdb
from os.path import dirname, realpath
#sys.path.append(dirname(dirname(dirname(realpath(__file__)))))

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=20)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)    # legend fontsize
plt.rc('figure', titlesize=24)  # fontsize of the figure title

def plot_calibration(args, test_preds, month=6):
    independent_eval = args.independent_eval
    # pdb.set_trace()
    probs_for_eval, golds_for_eval = get_probs_golds(
        args, test_preds, month, independent_eval)
    prob_true, prob_pred = compute_calibration_curve(golds_for_eval, probs_for_eval)
    plt.figure(figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))
    ax1.plot(prob_pred, prob_true, 's-')
    ax1.plot([0, 1], [0, 1], linestyle='--', 
             color='grey', label='random guess (AUROC: 0.500)')
    ax2.hist(probs_for_eval, bins=10, range=(0, 1))
    ax1.set_xlim([-0.01, 1.01])
    ax1.set_ylim([-0.01, 1.01])
    ax1.set_xlabel('Predicted value')
    ax1.set_ylabel('Fraction of positive')
    ax1.set_title("Calibration plot")
    ax2.set_xlabel('Prediction probability')
    ax2.set_ylabel('Count')
    plt.tight_layout()



In [ ]:
metrics = ['auroc', 'auprc', 'loss']
result_path_pat_3m = 'logs_training_transformer_va3m/{}.results'
result_path_pat_1m = 'logs_training_transformer_va1m_part0/{}.results'

# -----------------------------------------------------------------------
# --- Non-random sampling combinded with learning rate and weight decay -
# -----------------------------------------------------------------------

# The first model trained from full VA dataset
result_path = 'logs/all_epoch_10_w1_batch_12816_devlen10.results'
params = get_params(result_path)
test_path = 'logs/all_epoch_10_w1_batch_12816_devlen10.results.test_stats'
print_stats(test_path)
epoch_path = 'logs/all_epoch_10_w1_batch_12816_devlen10.results.epoch_stats'
with open(epoch_path, 'rb') as f:
    epochs = pkl.load(f)

In [210]:
x0= np.zeros( shape = [len(Df),2])

In [211]:
x0[:,0] = Df['probs'].to_numpy()

In [212]:
#m_pred = np.array(Df['probs'].to_numpy(), np.zeros(len(Df)))

In [213]:
y = np.array(Df['golds'])

In [214]:
import SimpleTemp

In [215]:
from SimpleTemp import _temperature_scaling

In [216]:
M1 = M.drop_duplicates(['patient_id'])

In [217]:
m0 = M.duplicated(['patient_id'])
M234 = M[m0]
M2 = M234.drop_duplicates(['patient_id'])

In [218]:
m00 = M234.duplicated(['patient_id'])
M34 = M234[m00]
M3 = M34.drop_duplicates(['patient_id'])

In [219]:
m000 = M34.duplicated(['patient_id'])
M4 = M34[m000]

In [220]:
z1 = np.array(M1['golds'],dtype = int)

x1_0= np.zeros( shape = [len(M1),2])
x1_0[:,0] = M1['probs'].to_numpy()

cal_1 = temperature_scaling((x1_0), z1,.1)  
cal_1**.5
    

0.25061632413479074

In [221]:
z2 = np.array(M2['golds'],dtype = int)
x2_0= np.zeros( shape = [len(M2),2])  
x2_0[:,0] = M2['probs'].to_numpy()
cal_2 = temperature_scaling((x2_0), z2, .02)  
cal_2**.5

0.17200214512706363

In [222]:
z3 = np.array(M3['golds'],dtype = int)
x3_0= np.zeros( shape = [len(M3),2])  
x3_0[:,0] = M3['probs'].to_numpy()
cal_3 = temperature_scaling((x3_0), z3, .02)  
cal_3**.5

0.1249453878671216

In [223]:
z4 = np.array(M4['golds'],dtype = int)
x4_0= np.zeros( shape = [len(M4),2])  
x4_0[:,0] = M4['probs'].to_numpy()
cal_4 = temperature_scaling((x4_0), z4, .02)  
cal_4**.5

0.11373210780318808

In [224]:
cal_4

0.012934992345355995

In [71]:
x1_0

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [72]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
SVC_classifier = SVC(probability=True)

In [90]:
SVC_classifier.fit(X_train, y_train)
svc_predictions = SVC_classifier.predict_proba(X_train)
Logistic_Regression = LogisticRegression()
Logistic_Regression.fit(X_train, y_train)
logistic_predictions = Logistic_Regression.predict_proba(X_train)
temp = temperature_scaling(svc_predictions, y_train, 1.5)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:53: RuntimeWarning: divide by zero encountered in log
  losses = np.log(losses)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:29: RuntimeWarning: overflow encountered in exp
  exp_T_output = np.exp(exp_T_output)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:74: RuntimeWarning: invalid value encountered in add
  dL_dts: np.ndarray = (term_1 + term_2) / exp_T_sum
D:\Programs\Anaconda3\Lib\site-packages\scipy\special\_logsumexp.py:224: RuntimeWarning: invalid value encountered in subtract
  exp_x_shifted = np.exp(x - x_max)


In [91]:
temp2 = temperature_scaling(logistic_predictions, y_train, 1.5)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:29: RuntimeWarning: overflow encountered in exp
  exp_T_output = np.exp(exp_T_output)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:74: RuntimeWarning: invalid value encountered in add
  dL_dts: np.ndarray = (term_1 + term_2) / exp_T_sum
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_3996\3620560755.py:53: RuntimeWarning: divide by zero encountered in log
  losses = np.log(losses)


In [92]:
temp2

1.118859322679242

In [93]:
temp

1.1200943073833476

In [64]:
y_train

array([0, 0, 0, 1, 2, 0, 0, 1, 1, 2, 1, 2, 0, 2, 0, 2, 0, 1, 1, 1, 2, 0,
       0, 1, 1, 1, 2, 2, 2, 1, 0, 1, 0, 2, 2, 2, 0, 1, 1, 1, 0, 1, 2, 0,
       1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 2, 0, 2, 1, 2, 0, 1, 2, 1, 2, 0, 0,
       1, 2, 0, 1, 0, 2, 0, 1, 0, 2, 1, 2, 2, 1, 2, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1, 0, 2, 1, 0, 2, 1, 2, 1, 2,
       2, 0])

In [5]:
outpath = os.path.join(src_path ,'Notebooks//PostRun_Analysis//output/Multi', "J10DX.csv")


In [66]:
np.shape(y_train)

(112,)

In [65]:
np.shape(svc_predictions)

(112, 3)

In [ ]:
svc_predictions

In [ ]:
logistic_predictions

In [95]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [94]:
 logistic_predictions

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan

In [ ]:
 #M.to_csv(outpath) 


In [11]:
M = pd.merge(g6, testDF)
m = M[["patient_id","dx_date", "index_date", "model_date",  "outcome","outcome_date","p6" ]]
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])

#### THE PURPOSE OF THIS STEP IS TO ACTUALLY REPLACE THE 1's after 2 years with 0's! NEED it, in dxdate this is NOT updated
m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_84892\383890625.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_84892\383890625.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_84892\383890625.py:14: SettingWithCopyWarning: 
A value is trying to be se

In [22]:
sklearn.metrics.roc_auc_score(m1['outcome'],  m1['p6'], average = 'samples')


0.5600783435183764

In [13]:
( confusion_matrix (m1['outcome'],  m1['p6']>.5) )

array([[72681,     5],
       [ 4301,     0]], dtype=int64)

In [79]:
# how many people had event within 6 months of index date

In [27]:
m1.to_csv('output/XCoh_Sen.csv') 

In [27]:
m1.to_csv('output/XCoh_Sen.csv') 

In [34]:
#m1.to_excel('output/dxMD_Sensitivity90.xlsx') 

In [35]:
statistics.quantiles( M['len180'],n=20)

KeyError: 'len180'

In [23]:
plt.hist(M['len180'])

(array([2.2715e+04, 1.8050e+03, 1.3000e+02, 2.5000e+01, 1.2000e+01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 2.0000e+00]),
 array([1.00000e+00, 2.37810e+03, 4.75520e+03, 7.13230e+03, 9.50940e+03,
        1.18865e+04, 1.42636e+04, 1.66407e+04, 1.90178e+04, 2.13949e+04,
        2.37720e+04]),
 <BarContainer object of 10 artists>)

In [10]:
m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]

m1 = pd.concat([m_VE1, m_VE0, m0])

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m['d']= pd.to_datetime(m['outcome_date']) - pd.to_datetime(m['model_date'])
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_73432\1057840980.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
